In [31]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [32]:
# Extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

#Extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

#Extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

#Extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [33]:

if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=rtx+4080&crid=3CQPF29XHLHXY&sprefix=rtx+4080+%2Caps%2C317&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [34]:
amazon_df

,title,price,rating,reviews,availability
0,ASUS TUF Gaming NVIDIA GeForce RTX™ 4080 Super...,,4.5 out of 5 stars,478 ratings,
1,MSI Gaming RTX 4080 Super 16G Expert Graphics ...,,5.0 out of 5 stars,4 ratings,
2,MSI Gaming RTX 4080 Super 16G Ventus 3X OC Gra...,,4.0 out of 5 stars,4 ratings,
3,ZOTAC Gaming GeForce RTX 4080 Super Trinity OC...,,5.0 out of 5 stars,4 ratings,
4,PNY GeForce RTX™ 4080 Super 16GB Verto™ Overcl...,,3.9 out of 5 stars,52 ratings,Not Available
5,MSI Gaming RTX 4080 Super 16G Gaming X Slim Gr...,,3.3 out of 5 stars,7 ratings,In Stock
6,GIGABYTE GeForce RTX 4080 Super AERO OC 16G Gr...,,3.0 out of 5 stars,2 ratings,
7,MSI Gaming GeForce RTX 4080 16GB GDRR6X 256-Bi...,,4.5 out of 5 stars,88 ratings,Only 3 left in stock - order soon.
8,ASUS ROG Strix GeForce RTX™ 4070 OC Edition Ga...,,4.7 out of 5 stars,491 ratings,In Stock
9,ZOTAC Gaming GeForce RTX 4080 16GB Trinity OC ...,,4.4 out of 5 stars,208 ratings,Only 4 left in stock - order soon.
